# 06.6 - Comparación Completa de TODOS los Modelos

Este notebook consolida y compara TODOS los modelos evaluados en los notebooks 05, 05.1, 05.2, 05.3, 05.4 y 05.5. Voy a cargar todos los resultados, hacer análisis comparativos profundos con gráficas descriptivas, y analizar qué parámetros/features prioriza cada tipo de modelo.

**Objetivo:** Entender cuáles son los mejores modelos globalmente y qué características los hacen exitosos.

In [1]:
# Importaciones completas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import json
import os
import warnings
warnings.filterwarnings('ignore')

# Plotly para visualizaciones interactivas
import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
import plotly.colors as colors

# Para análisis estadístico
from scipy import stats
import seaborn as sns

print(f"Notebook 05.6 - Comparación Completa iniciado: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("Voy a consolidar y comparar TODOS los modelos evaluados")

Notebook 05.6 - Comparación Completa iniciado: 2025-08-21 01:00:53
Voy a consolidar y comparar TODOS los modelos evaluados


In [2]:
# Cargar todos los resultados de los notebooks anteriores
print("Cargando resultados de todos los notebooks...")

# Diccionario para almacenar todos los resultados
todos_los_resultados = {}
archivos_resultados = {
    'Boosting': '../models/resultados_boosting_winsorized.json',
    'Redes_Neuronales': '../models/resultados_redes_neuronales_winsorized.json', 
    'SVM': '../models/resultados_svm_winsorized.json',
    'Modelos_Lineales': '../models/resultados_modelos_lineales_winsorized.json',
    'Ensemble': '../models/resultados_ensemble_winsorized.json',
    'Otros_Algoritmos': '../models/resultados_otros_algoritmos_winsorized.json'
}

# Cargar cada archivo
for categoria, archivo in archivos_resultados.items():
    if os.path.exists(archivo):
        with open(archivo, 'r') as f:
            resultados = json.load(f)
            # Agregar categoría a cada modelo
            for modelo, datos in resultados.items():
                datos['categoria'] = categoria
                todos_los_resultados[f"{categoria}_{modelo}"] = datos
        print(f"+ Cargado {categoria}: {len(resultados)} modelos")
    else:
        print(f"* No encontrado: {archivo}")

print(f"\n- Total modelos cargados: {len(todos_los_resultados)}")

# Verificar estructura de datos
if todos_los_resultados:
    primer_modelo = list(todos_los_resultados.keys())[0]
    print(f"\nEstructura ejemplo ({primer_modelo}):")
    for key in todos_los_resultados[primer_modelo].keys():
        print(f"  - {key}")

Cargando resultados de todos los notebooks...
+ Cargado Boosting: 11 modelos
+ Cargado Redes_Neuronales: 12 modelos
+ Cargado SVM: 5 modelos
+ Cargado Modelos_Lineales: 21 modelos
+ Cargado Ensemble: 14 modelos
+ Cargado Otros_Algoritmos: 20 modelos

- Total modelos cargados: 83

Estructura ejemplo (Boosting_XGBoost_Conservative):
  - nombre
  - timestamp
  - parametros
  - tiempo_entrenamiento
  - metricas_test
  - cv_scores
  - predicciones
  - categoria


In [3]:
# Convertir a DataFrame para análisis
print("Creando DataFrame consolidado...")

datos_consolidados = []

for modelo_id, datos in todos_los_resultados.items():
    # Extraer información básica
    fila = {
        'Modelo_ID': modelo_id,
        'Nombre': datos.get('nombre', modelo_id),
        'Categoria': datos.get('categoria', 'Unknown'),
        'Tipo_Algoritmo': datos.get('tipo_modelo', datos.get('tipo_svm', datos.get('tipo_algoritmo', datos.get('tipo_ensemble', 'Unknown')))),
        'Tiempo_Entrenamiento': datos.get('tiempo_entrenamiento', 0),
        'Usar_Escalado': datos.get('usar_escalado', False)
    }
    
    # Extraer métricas
    metricas = datos.get('metricas', {})
    fila.update({
        'Accuracy': metricas.get('accuracy', 0),
        'Precision': metricas.get('precision', 0),
        'Recall': metricas.get('recall', 0),
        'F1_Score': metricas.get('f1', 0),
        'ROC_AUC': metricas.get('roc_auc', 0) or 0
    })
    
    # Cross-validation
    fila.update({
        'CV_F1_Mean': datos.get('cv_f1_mean', 0),
        'CV_F1_Std': datos.get('cv_f1_std', 0)
    })
    
    # Información específica por tipo de modelo
    fila.update({
        'N_Estimators': datos.get('n_estimators', datos.get('n_estimadores', 0)) or 0,
        'Support_Vectors': datos.get('n_support_vectors', 0) or 0,
        'Features_Activos': datos.get('features_activos', 0) or 0,
        'Hidden_Layers': datos.get('hidden_layers', 0) or 0,
        'N_Neighbors': datos.get('n_neighbors', 0) or 0
    })
    
    # Parámetros importantes (convertir a string para análisis)
    parametros = datos.get('parametros', {})
    fila['Parametros_JSON'] = json.dumps(parametros) if parametros else '{}'
    
    datos_consolidados.append(fila)

# Crear DataFrame
df_completo = pd.DataFrame(datos_consolidados)

print(f"DataFrame creado: {len(df_completo)} modelos x {len(df_completo.columns)} columnas")
print(f"\nCategorías encontradas: {df_completo['Categoria'].value_counts().to_dict()}")
print(f"\nRango F1-Score: {df_completo['F1_Score'].min():.4f} - {df_completo['F1_Score'].max():.4f}")
print(f"Rango Tiempo: {df_completo['Tiempo_Entrenamiento'].min():.3f}s - {df_completo['Tiempo_Entrenamiento'].max():.2f}s")

Creando DataFrame consolidado...
DataFrame creado: 83 modelos x 19 columnas

Categorías encontradas: {'Modelos_Lineales': 21, 'Otros_Algoritmos': 20, 'Ensemble': 14, 'Redes_Neuronales': 12, 'Boosting': 11, 'SVM': 5}

Rango F1-Score: 0.0000 - 0.7021
Rango Tiempo: 0.000s - 129.82s


## Análisis Global de Rendimiento

Primero veo el panorama general de todos los modelos.

In [4]:
# Ranking global de modelos
print("\n" + "="*100)
print("RANKING GLOBAL - TOP 20 MODELOS POR F1-SCORE")
print("="*100)

df_ranking = df_completo.sort_values('F1_Score', ascending=False).head(20)

for i, (_, row) in enumerate(df_ranking.iterrows(), 1):
    tiempo_info = f"{row['Tiempo_Entrenamiento']:.2f}s"
    escalado_info = "(Scaled)" if row['Usar_Escalado'] else ""
    categoria_info = f"[{row['Categoria']}]"
    
    print(f"{i:2d}. {row['Nombre']:<35} F1: {row['F1_Score']:.4f} | "
          f"{categoria_info:<20} | {tiempo_info:<8} {escalado_info}")

# Campeón absoluto
campeon = df_ranking.iloc[0]
print(f"\n- CAMPEÓN ABSOLUTO: {campeon['Nombre']}")
print(f"   F1-Score: {campeon['F1_Score']:.4f}")
print(f"   Categoría: {campeon['Categoria']}")
print(f"   Tipo: {campeon['Tipo_Algoritmo']}")
print(f"   Tiempo: {campeon['Tiempo_Entrenamiento']:.3f}s")
print(f"   ROC-AUC: {campeon['ROC_AUC']:.4f}")
print(f"   Accuracy: {campeon['Accuracy']:.4f}")


RANKING GLOBAL - TOP 20 MODELOS POR F1-SCORE
 1. NN High Regularization (100,50)     F1: 0.7021 | [Redes_Neuronales]   | 33.04s   
 2. NN Two Symmetric (80,80)            F1: 0.6969 | [Redes_Neuronales]   | 21.39s   
 3. NN Deep Uniform (100,100,100)       F1: 0.6916 | [Redes_Neuronales]   | 47.33s   
 4. NN Deep Decreasing (150,100,50)     F1: 0.6909 | [Redes_Neuronales]   | 65.59s   
 5. NN Small (100)                      F1: 0.6887 | [Redes_Neuronales]   | 12.18s   
 6. NN Diamond (80,120,80)              F1: 0.6869 | [Redes_Neuronales]   | 40.26s   
 7. NN Tiny (50)                        F1: 0.6859 | [Redes_Neuronales]   | 14.14s   
 8. Gradient Boosting (100, depth=4)    F1: 0.6836 | [Ensemble]           | 58.50s   
 9. NN SGD Momentum (100,50)            F1: 0.6826 | [Redes_Neuronales]   | 129.82s  
10. NN Bottleneck (120,30)              F1: 0.6823 | [Redes_Neuronales]   | 22.39s   
11. Voting Trees (RF+XGB+LGB+ET)        F1: 0.6804 | [Ensemble]           | 21.04s   
12. NN T

In [5]:
# Gráfica de ranking global
fig_ranking = px.bar(
    df_ranking,
    x='F1_Score',
    y='Nombre',
    color='Categoria',
    title='- TOP 20 MODELOS - Ranking Global por F1-Score',
    orientation='h',
    color_discrete_sequence=px.colors.qualitative.Set3
)

fig_ranking.update_layout(
    height=800,
    yaxis={'categoryorder': 'total ascending'},
    font=dict(size=12),
    title_font=dict(size=16)
)

fig_ranking.show()

# Distribución de rendimiento por categoría
fig_box = px.box(
    df_completo,
    x='Categoria',
    y='F1_Score',
    title='- Distribución de F1-Score por Categoría de Modelo',
    color='Categoria',
    points='all'
)

fig_box.update_layout(
    height=600,
    xaxis_tickangle=-45
)

fig_box.show()

## Análisis de Eficiencia: Tiempo vs Rendimiento

Analizo la relación entre tiempo de entrenamiento y rendimiento.

In [6]:
# Scatter plot: Tiempo vs F1-Score
fig_efficiency = px.scatter(
    df_completo,
    x='Tiempo_Entrenamiento',
    y='F1_Score',
    color='Categoria',
    size='ROC_AUC',
    hover_data=['Nombre', 'Accuracy'],
    title='- Eficiencia de Modelos: Tiempo vs F1-Score (tamaño = ROC-AUC)',
    log_x=True  # Escala logarítmica para tiempo
)

fig_efficiency.update_layout(height=600)
fig_efficiency.show()

# Análisis de eficiencia: mejores modelos por tiempo
print("\n- ANÁLISIS DE EFICIENCIA")
print("="*50)

# Modelos rápidos (< 1 segundo) con buen rendimiento
rapidos_buenos = df_completo[
    (df_completo['Tiempo_Entrenamiento'] < 1.0) & 
    (df_completo['F1_Score'] > df_completo['F1_Score'].quantile(0.75))
].sort_values('F1_Score', ascending=False)

print(f"\n- MODELOS RÁPIDOS Y BUENOS (< 1s, F1 > Q3):")
for _, row in rapidos_buenos.head(10).iterrows():
    print(f"   {row['Nombre']:<30} F1: {row['F1_Score']:.4f} | {row['Tiempo_Entrenamiento']:.3f}s")

# Calcular ratio eficiencia (F1/tiempo)
df_completo['Ratio_Eficiencia'] = df_completo['F1_Score'] / (df_completo['Tiempo_Entrenamiento'] + 0.001)  # +0.001 para evitar división por 0

print(f"\n- TOP 10 POR RATIO EFICIENCIA (F1/Tiempo):")
top_eficiencia = df_completo.sort_values('Ratio_Eficiencia', ascending=False).head(10)
for _, row in top_eficiencia.iterrows():
    print(f"   {row['Nombre']:<30} Ratio: {row['Ratio_Eficiencia']:.1f} | F1: {row['F1_Score']:.4f} | {row['Tiempo_Entrenamiento']:.3f}s")


- ANÁLISIS DE EFICIENCIA

- MODELOS RÁPIDOS Y BUENOS (< 1s, F1 > Q3):
   KNN k=5 Manhattan Distance     F1: 0.6666 | 0.061s
   Quadratic Discriminant Analysis F1: 0.6662 | 0.532s
   Naive Bayes Gaussian Low Smooth F1: 0.6662 | 0.087s
   KNN k=15 Distance              F1: 0.6655 | 0.037s

- TOP 10 POR RATIO EFICIENCIA (F1/Tiempo):
   KNN k=5 Uniform                Ratio: 28.4 | F1: 0.6551 | 0.022s
   Naive Bayes Complement         Ratio: 27.7 | F1: 0.6147 | 0.021s
   Naive Bayes Multinomial        Ratio: 27.6 | F1: 0.6407 | 0.022s
   KNN k=10 Uniform               Ratio: 18.5 | F1: 0.6357 | 0.033s
   KNN k=15 Distance              Ratio: 17.4 | F1: 0.6655 | 0.037s
   KNN k=3 Distance Weight        Ratio: 14.9 | F1: 0.6474 | 0.042s
   KNN k=5 Manhattan Distance     Ratio: 10.7 | F1: 0.6666 | 0.061s
   Naive Bayes Gaussian Low Smooth Ratio: 7.6 | F1: 0.6662 | 0.087s
   Extra Tree Single (depth=12)   Ratio: 7.6 | F1: 0.6371 | 0.083s
   Naive Bayes Bernoulli          Ratio: 5.9 | F1: 0.616

## Análisis Profundo por Categoría

Analizo cada categoría de modelos en detalle.

In [7]:
# Estadísticas por categoría
print("\nESTADÍSTICAS DETALLADAS POR CATEGORÍA")
print("="*80)

stats_categoria = df_completo.groupby('Categoria').agg({
    'F1_Score': ['count', 'mean', 'std', 'min', 'max'],
    'Tiempo_Entrenamiento': ['mean', 'std', 'min', 'max'],
    'ROC_AUC': ['mean', 'max'],
    'Accuracy': ['mean', 'max']
}).round(4)

for categoria in stats_categoria.index:
    print(f"\n- {categoria}:")
    print(f"   Modelos: {int(stats_categoria.loc[categoria, ('F1_Score', 'count')])}")
    print(f"   F1 promedio: {stats_categoria.loc[categoria, ('F1_Score', 'mean')]:.4f} ± {stats_categoria.loc[categoria, ('F1_Score', 'std')]:.4f}")
    print(f"   F1 rango: [{stats_categoria.loc[categoria, ('F1_Score', 'min')]:.4f} - {stats_categoria.loc[categoria, ('F1_Score', 'max')]:.4f}]")
    print(f"   Tiempo promedio: {stats_categoria.loc[categoria, ('Tiempo_Entrenamiento', 'mean')]:.3f}s ± {stats_categoria.loc[categoria, ('Tiempo_Entrenamiento', 'std')]:.3f}s")
    print(f"   ROC-AUC promedio: {stats_categoria.loc[categoria, ('ROC_AUC', 'mean')]:.4f}")
    
    # Mejor modelo de esta categoría
    mejor_categoria = df_completo[df_completo['Categoria'] == categoria].sort_values('F1_Score', ascending=False).iloc[0]
    print(f"   - Mejor: {mejor_categoria['Nombre']} (F1: {mejor_categoria['F1_Score']:.4f})")

# Heatmap de rendimiento por categoría
categoria_metrics = df_completo.groupby('Categoria')[['F1_Score', 'Accuracy', 'ROC_AUC', 'Precision', 'Recall']].mean()

fig_heatmap = px.imshow(
    categoria_metrics.T,
    title='- Heatmap de Rendimiento Promedio por Categoría',
    color_continuous_scale='RdYlBu_r',
    aspect='auto'
)

fig_heatmap.update_layout(
    height=400,
    xaxis_title='Categoría de Modelo',
    yaxis_title='Métrica'
)

fig_heatmap.show()


ESTADÍSTICAS DETALLADAS POR CATEGORÍA

- Boosting:
   Modelos: 11
   F1 promedio: 0.0000 ± 0.0000
   F1 rango: [0.0000 - 0.0000]
   Tiempo promedio: 9.999s ± 24.824s
   ROC-AUC promedio: 0.0000
   - Mejor: XGBoost Conservative (F1: 0.0000)

- Ensemble:
   Modelos: 14
   F1 promedio: 0.6622 ± 0.0100
   F1 rango: [0.6503 - 0.6836]
   Tiempo promedio: 17.974s ± 19.575s
   ROC-AUC promedio: 0.6562
   - Mejor: Gradient Boosting (100, depth=4) (F1: 0.6836)

- Modelos_Lineales:
   Modelos: 21
   F1 promedio: 0.6324 ± 0.0360
   F1 rango: [0.5640 - 0.6570]
   Tiempo promedio: 7.853s ± 23.329s
   ROC-AUC promedio: 0.6610
   - Mejor: Logistic Regression High Reg (C=0.1) (F1: 0.6570)

- Otros_Algoritmos:
   Modelos: 20
   F1 promedio: 0.6443 ± 0.0175
   F1 rango: [0.6147 - 0.6666]
   Tiempo promedio: 1.975s ± 5.144s
   ROC-AUC promedio: 0.6813
   - Mejor: KNN k=5 Manhattan Distance (F1: 0.6666)

- Redes_Neuronales:
   Modelos: 12
   F1 promedio: 0.6868 ± 0.0077
   F1 rango: [0.6764 - 0.7021]
   T

## Análisis de Características de Modelos

Analizo qué características técnicas influyen en el rendimiento.

In [8]:
# Análisis de escalado
print("\nANÁLISIS DE CARACTERÍSTICAS TÉCNICAS")
print("="*60)

# Impacto del escalado
escalado_stats = df_completo.groupby('Usar_Escalado')['F1_Score'].agg(['count', 'mean', 'std']).round(4)
print("\n- Impacto del Escalado de Datos:")
print(escalado_stats)

# Modelos con diferentes números de estimators
modelos_ensemble = df_completo[df_completo['N_Estimators'] > 0]
if len(modelos_ensemble) > 0:
    print(f"\n- Modelos Ensemble (con estimators): {len(modelos_ensemble)}")
    print(f"   Estimators promedio: {modelos_ensemble['N_Estimators'].mean():.1f}")
    print(f"   F1 promedio: {modelos_ensemble['F1_Score'].mean():.4f}")
    
    # Correlación entre n_estimators y rendimiento
    corr_estimators = modelos_ensemble['N_Estimators'].corr(modelos_ensemble['F1_Score'])
    print(f"   Correlación estimators-F1: {corr_estimators:.3f}")

# Análisis de complejidad vs rendimiento
fig_complexity = make_subplots(
    rows=2, cols=2,
    subplot_titles=[
        'N_Estimators vs F1-Score',
        'Support Vectors vs F1-Score', 
        'Features Activos vs F1-Score',
        'Tiempo vs F1-Score por Escalado'
    ]
)

# N_Estimators vs F1
ensemble_data = df_completo[df_completo['N_Estimators'] > 0]
if len(ensemble_data) > 0:
    fig_complexity.add_trace(
        go.Scatter(
            x=ensemble_data['N_Estimators'],
            y=ensemble_data['F1_Score'],
            mode='markers',
            text=ensemble_data['Nombre'],
            name='Ensemble'
        ),
        row=1, col=1
    )

# Support Vectors vs F1
svm_data = df_completo[df_completo['Support_Vectors'] > 0]
if len(svm_data) > 0:
    fig_complexity.add_trace(
        go.Scatter(
            x=svm_data['Support_Vectors'],
            y=svm_data['F1_Score'],
            mode='markers',
            text=svm_data['Nombre'],
            name='SVM'
        ),
        row=1, col=2
    )

# Features Activos vs F1
linear_data = df_completo[df_completo['Features_Activos'] > 0]
if len(linear_data) > 0:
    fig_complexity.add_trace(
        go.Scatter(
            x=linear_data['Features_Activos'],
            y=linear_data['F1_Score'],
            mode='markers',
            text=linear_data['Nombre'],
            name='Lineales'
        ),
        row=2, col=1
    )

# Tiempo por escalado
for escalado in [True, False]:
    subset = df_completo[df_completo['Usar_Escalado'] == escalado]
    fig_complexity.add_trace(
        go.Scatter(
            x=subset['Tiempo_Entrenamiento'],
            y=subset['F1_Score'],
            mode='markers',
            name=f'Escalado: {escalado}',
            text=subset['Nombre']
        ),
        row=2, col=2
    )

fig_complexity.update_layout(
    height=800,
    title_text='🔬 Análisis de Complejidad vs Rendimiento',
    showlegend=True
)

fig_complexity.show()


ANÁLISIS DE CARACTERÍSTICAS TÉCNICAS

- Impacto del Escalado de Datos:
               count    mean     std
Usar_Escalado                       
False             65  0.5419  0.2482
True              18  0.6523  0.0137

- Modelos Ensemble (con estimators): 14
   Estimators promedio: 59.3
   F1 promedio: 0.6622
   Correlación estimators-F1: 0.079


## Análisis de Parámetros e Importancian (IGNORAR)

Esta sección incluye código reutilizable para analizar qué parámetros prioriza cada modelo por si quiero luego usarlo en los otros notebooks de cada uno


In [9]:
def analizar_importancia_modelo(modelo_entrenado, X_test, y_test, nombre_modelo):
    """
    Analiza qué características/parámetros prioriza un modelo específico.
    Funciona con la mayoría de modelos de sklearn.
    """
    import numpy as np
    import pandas as pd
    import plotly.express as px
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots
    
    print(f"\n- ANÁLISIS DE IMPORTANCIA - {nombre_modelo}")
    print("-" * 60)
    
    info_modelo = {
        'nombre': nombre_modelo,
        'tipo': type(modelo_entrenado).__name__,
        'parametros': modelo_entrenado.get_params()
    }
    
    # 1. Feature Importance (si está disponible)
    if hasattr(modelo_entrenado, 'feature_importances_'):
        importances = modelo_entrenado.feature_importances_
        feature_names = [f'Feature_{i}' for i in range(len(importances))]
        
        # Top 15 features más importantes
        top_indices = np.argsort(importances)[-15:]
        top_importances = importances[top_indices]
        top_names = [feature_names[i] for i in top_indices]
        
        print(f" Feature Importances disponibles")
        print(f"  Top 5 features: {top_names[-5:]}")
        print(f"  Importancias: {top_importances[-5:]}")
        print(f"  Importancia media: {np.mean(top_importances):.4f}")
        # Gráfica de importancias
        fig_imp = px.bar(
            x=top_importances,
            y=top_names,
            orientation='h',
            title=f'Top 15 Feature Importances - {nombre_modelo}'
        )
        fig_imp.update_layout(yaxis={'categoryorder': 'total ascending'})
        fig_imp.show()
        
        info_modelo['feature_importance_disponible'] = True
        info_modelo['top_features'] = top_names[-5:]
        
    # 2. Coeficientes (modelos lineales)
    elif hasattr(modelo_entrenado, 'coef_'):
        coefs = modelo_entrenado.coef_[0] if len(modelo_entrenado.coef_.shape) > 1 else modelo_entrenado.coef_
        feature_names = [f'Feature_{i}' for i in range(len(coefs))]
        
        # Top coeficientes por magnitud absoluta
        top_indices = np.argsort(np.abs(coefs))[-15:]
        top_coefs = coefs[top_indices]
        top_names = [feature_names[i] for i in top_indices]
        
        print(f"Coeficientes disponibles")
        print(f"  Top 5 coef (absoluto): {top_names[-5:]}")
        print(f"  Valores: {top_coefs[-5:]}")
        
        # Gráfica de coeficientes
        fig_coef = px.bar(
            x=top_coefs,
            y=top_names,
            orientation='h',
            title=f'Top 15 Coeficientes - {nombre_modelo}',
            color=top_coefs,
            color_continuous_scale='RdBu_r'
        )
        fig_coef.update_layout(yaxis={'categoryorder': 'total ascending'})
        fig_coef.show()
        
        info_modelo['coeficientes_disponibles'] = True
        info_modelo['coef_max_abs'] = np.max(np.abs(coefs))
        info_modelo['coef_mean_abs'] = np.mean(np.abs(coefs))
        
    # 3. Información específica del modelo
    if hasattr(modelo_entrenado, 'n_estimators'):
        print(f"  N_estimators: {modelo_entrenado.n_estimators}")
        info_modelo['n_estimators'] = modelo_entrenado.n_estimators
        
    if hasattr(modelo_entrenado, 'max_depth'):
        print(f"  Max_depth: {modelo_entrenado.max_depth}")
        info_modelo['max_depth'] = modelo_entrenado.max_depth
        
    if hasattr(modelo_entrenado, 'C'):
        print(f"  Parámetro C: {modelo_entrenado.C}")
        info_modelo['C'] = modelo_entrenado.C
        
    if hasattr(modelo_entrenado, 'alpha'):
        print(f"  Parámetro alpha: {modelo_entrenado.alpha}")
        info_modelo['alpha'] = modelo_entrenado.alpha
        
    if hasattr(modelo_entrenado, 'learning_rate'):
        print(f"  Learning rate: {modelo_entrenado.learning_rate}")
        info_modelo['learning_rate'] = modelo_entrenado.learning_rate
        
    if hasattr(modelo_entrenado, 'n_neighbors'):
        print(f"  N_neighbors: {modelo_entrenado.n_neighbors}")
        info_modelo['n_neighbors'] = modelo_entrenado.n_neighbors
    
    # 4. Análisis de parámetros clave
    print(f"\n- Parámetros del modelo:")
    parametros_importantes = {
        'regularization': ['C', 'alpha', 'l1_ratio', 'penalty'],
        'complexity': ['max_depth', 'n_estimators', 'n_neighbors', 'hidden_layer_sizes'],
        'learning': ['learning_rate', 'solver', 'algorithm', 'kernel'],
        'sampling': ['subsample', 'max_samples', 'bootstrap']
    }
    
    for categoria, params in parametros_importantes.items():
        valores_encontrados = {}
        for param in params:
            if hasattr(modelo_entrenado, param):
                valores_encontrados[param] = getattr(modelo_entrenado, param)
        
        if valores_encontrados:
            print(f"  {categoria.capitalize()}: {valores_encontrados}")
            info_modelo[f'params_{categoria}'] = valores_encontrados
    
    return info_modelo

# EJEMPLO DE USO:
# info = analizar_importancia_modelo(modelo_entrenado, X_test, y_test, "Modelo")
# print(json.dumps(info, indent=2, default=str))


In [10]:
# Análisis de parámetros en los modelos actuales
print("\nANÁLISIS DE PARÁMETROS DE LOS MEJORES MODELOS")
print("="*70)

# Analizar top 10 modelos
top_10 = df_completo.sort_values('F1_Score', ascending=False).head(10)

parametros_analysis = []

for _, modelo in top_10.iterrows():
    try:
        parametros = json.loads(modelo['Parametros_JSON'])
        
        # Extraer parámetros clave
        info_parametros = {
            'Modelo': modelo['Nombre'],
            'F1_Score': modelo['F1_Score'],
            'Categoria': modelo['Categoria']
        }
        
        # Parámetros de regularización
        for param in ['C', 'alpha', 'l1_ratio', 'penalty']:
            if param in parametros:
                info_parametros[f'reg_{param}'] = parametros[param]
        
        # Parámetros de complejidad
        for param in ['max_depth', 'n_estimators', 'n_neighbors', 'hidden_layer_sizes']:
            if param in parametros:
                info_parametros[f'comp_{param}'] = parametros[param]
        
        # Parámetros de aprendizaje
        for param in ['learning_rate', 'solver', 'kernel', 'algorithm']:
            if param in parametros:
                info_parametros[f'learn_{param}'] = parametros[param]
        
        parametros_analysis.append(info_parametros)
        
    except Exception as e:
        print(f"Error analizando {modelo['Nombre']}: {e}")

# Mostrar análisis
for i, params in enumerate(parametros_analysis, 1):
    print(f"\n{i}. {params['Modelo']} (F1: {params['F1_Score']:.4f})")
    print(f"   Categoría: {params['Categoria']}")
    
    # Mostrar parámetros por tipo
    for tipo in ['reg', 'comp', 'learn']:
        params_tipo = {k.replace(f'{tipo}_', ''): v for k, v in params.items() if k.startswith(f'{tipo}_')}
        if params_tipo:
            tipo_nombre = {'reg': 'Regularización', 'comp': 'Complejidad', 'learn': 'Aprendizaje'}[tipo]
            print(f"   {tipo_nombre}: {params_tipo}")


ANÁLISIS DE PARÁMETROS DE LOS MEJORES MODELOS
Error analizando Gradient Boosting (100, depth=4): string indices must be integers, not 'str'

1. NN High Regularization (100,50) (F1: 0.7021)
   Categoría: Redes_Neuronales

2. NN Two Symmetric (80,80) (F1: 0.6969)
   Categoría: Redes_Neuronales

3. NN Deep Uniform (100,100,100) (F1: 0.6916)
   Categoría: Redes_Neuronales

4. NN Deep Decreasing (150,100,50) (F1: 0.6909)
   Categoría: Redes_Neuronales

5. NN Small (100) (F1: 0.6887)
   Categoría: Redes_Neuronales

6. NN Diamond (80,120,80) (F1: 0.6869)
   Categoría: Redes_Neuronales

7. NN Tiny (50) (F1: 0.6859)
   Categoría: Redes_Neuronales

8. NN SGD Momentum (100,50) (F1: 0.6826)
   Categoría: Redes_Neuronales

9. NN Bottleneck (120,30) (F1: 0.6823)
   Categoría: Redes_Neuronales


## Análisis de Patrones y Recomendaciones

Identifico patrones en los mejores modelos y genero recomendaciones.

In [11]:
# Análisis de patrones en mejores modelos
print("\nANÁLISIS DE PATRONES EN MEJORES MODELOS")
print("="*70)

# Top 20% de modelos
threshold_top = df_completo['F1_Score'].quantile(0.8)
mejores_modelos = df_completo[df_completo['F1_Score'] >= threshold_top]

print(f"Analizando top {len(mejores_modelos)} modelos (F1 >= {threshold_top:.4f})")

# Patrones por categoría
print("\n- Distribución de mejores modelos por categoría:")
distribucion_mejores = mejores_modelos['Categoria'].value_counts()
for categoria, count in distribucion_mejores.items():
    porcentaje = (count / len(mejores_modelos)) * 100
    print(f"   {categoria}: {count} modelos ({porcentaje:.1f}%)")

# Características comunes de mejores modelos
print("\n- Características comunes de mejores modelos:")
print(f"   Usan escalado: {mejores_modelos['Usar_Escalado'].mean()*100:.1f}%")
print(f"   Tiempo promedio: {mejores_modelos['Tiempo_Entrenamiento'].mean():.3f}s")
print(f"   ROC-AUC promedio: {mejores_modelos['ROC_AUC'].mean():.4f}")

# Comparar con modelos regulares
otros_modelos = df_completo[df_completo['F1_Score'] < threshold_top]
print(f"\n- Comparación con otros modelos:")
print(f"   Mejores - Tiempo promedio: {mejores_modelos['Tiempo_Entrenamiento'].mean():.3f}s")
print(f"   Otros - Tiempo promedio: {otros_modelos['Tiempo_Entrenamiento'].mean():.3f}s")
print(f"   Mejores - Usan escalado: {mejores_modelos['Usar_Escalado'].mean()*100:.1f}%")
print(f"   Otros - Usan escalado: {otros_modelos['Usar_Escalado'].mean()*100:.1f}%")

# Gráfica de comparación
fig_comparison = make_subplots(
    rows=2, cols=2,
    subplot_titles=[
        'Distribución F1-Score',
        'Tiempo de Entrenamiento', 
        'ROC-AUC Distribution',
        'Uso de Escalado'
    ]
)

# F1-Score distribution
fig_comparison.add_trace(
    go.Histogram(x=mejores_modelos['F1_Score'], name='Top 20%', opacity=0.7),
    row=1, col=1
)
fig_comparison.add_trace(
    go.Histogram(x=otros_modelos['F1_Score'], name='Otros 80%', opacity=0.7),
    row=1, col=1
)

# Tiempo distribution  
fig_comparison.add_trace(
    go.Histogram(x=mejores_modelos['Tiempo_Entrenamiento'], name='Top 20%', opacity=0.7, showlegend=False),
    row=1, col=2
)
fig_comparison.add_trace(
    go.Histogram(x=otros_modelos['Tiempo_Entrenamiento'], name='Otros 80%', opacity=0.7, showlegend=False),
    row=1, col=2
)

# ROC-AUC distribution
fig_comparison.add_trace(
    go.Histogram(x=mejores_modelos['ROC_AUC'], name='Top 20%', opacity=0.7, showlegend=False),
    row=2, col=1
)
fig_comparison.add_trace(
    go.Histogram(x=otros_modelos['ROC_AUC'], name='Otros 80%', opacity=0.7, showlegend=False),
    row=2, col=1
)

# Uso de escalado
escalado_mejores = mejores_modelos['Usar_Escalado'].value_counts()
escalado_otros = otros_modelos['Usar_Escalado'].value_counts()

fig_comparison.add_trace(
    go.Bar(x=['No Escalado', 'Escalado'], y=[escalado_mejores.get(False, 0), escalado_mejores.get(True, 0)], 
           name='Top 20%', opacity=0.7, showlegend=False),
    row=2, col=2
)
fig_comparison.add_trace(
    go.Bar(x=['No Escalado', 'Escalado'], y=[escalado_otros.get(False, 0), escalado_otros.get(True, 0)], 
           name='Otros 80%', opacity=0.7, showlegend=False),
    row=2, col=2
)

fig_comparison.update_layout(
    height=800,
    title_text='- Comparación: Mejores Modelos vs Resto'
)

fig_comparison.show()


ANÁLISIS DE PATRONES EN MEJORES MODELOS
Analizando top 17 modelos (F1 >= 0.6664)

- Distribución de mejores modelos por categoría:
   Redes_Neuronales: 12 modelos (70.6%)
   Ensemble: 4 modelos (23.5%)
   Otros_Algoritmos: 1 modelos (5.9%)

- Características comunes de mejores modelos:
   Usan escalado: 11.8%
   Tiempo promedio: 31.866s
   ROC-AUC promedio: 0.7446

- Comparación con otros modelos:
   Mejores - Tiempo promedio: 31.866s
   Otros - Tiempo promedio: 8.258s
   Mejores - Usan escalado: 11.8%
   Otros - Usan escalado: 24.2%


## Recomendaciones Finales

Genero recomendaciones basadas en todo el análisis.

In [12]:
# Generar recomendaciones finales
print("\n" + "="*100)
print("RECOMENDACIONES FINALES BASADAS EN ANÁLISIS COMPLETO")
print("="*100)

# Top 5 modelos absolutos
top_5_global = df_completo.sort_values('F1_Score', ascending=False).head(5)
print("\n- TOP 5 MODELOS RECOMENDADOS (rendimiento absoluto):")
for i, (_, modelo) in enumerate(top_5_global.iterrows(), 1):
    print(f"   {i}. {modelo['Nombre']} - F1: {modelo['F1_Score']:.4f} | ROC: {modelo['ROC_AUC']:.4f} | Tiempo: {modelo['Tiempo_Entrenamiento']:.3f}s")

# Top 5 por eficiencia
top_5_eficiencia = df_completo.sort_values('Ratio_Eficiencia', ascending=False).head(5)
print("\n- TOP 5 MODELOS EFICIENTES (mejor ratio rendimiento/tiempo):")
for i, (_, modelo) in enumerate(top_5_eficiencia.iterrows(), 1):
    print(f"   {i}. {modelo['Nombre']} - Ratio: {modelo['Ratio_Eficiencia']:.1f} | F1: {modelo['F1_Score']:.4f} | Tiempo: {modelo['Tiempo_Entrenamiento']:.3f}s")

# Recomendaciones por caso de uso
print("\n- RECOMENDACIONES POR CASO DE USO:")

# Para producción (balance rendimiento-velocidad)
prod_candidates = df_completo[
    (df_completo['F1_Score'] > df_completo['F1_Score'].quantile(0.85)) &
    (df_completo['Tiempo_Entrenamiento'] < df_completo['Tiempo_Entrenamiento'].median())
].sort_values('F1_Score', ascending=False)

print("\n- Para PRODUCCIÓN (buen rendimiento + rápido):")
for _, modelo in prod_candidates.head(3).iterrows():
    print(f"   • {modelo['Nombre']} - F1: {modelo['F1_Score']:.4f}, Tiempo: {modelo['Tiempo_Entrenamiento']:.3f}s")

# Para máximo rendimiento
print("\n- Para MÁXIMO RENDIMIENTO (sin restricción de tiempo):")
for _, modelo in top_5_global.head(3).iterrows():
    print(f"   • {modelo['Nombre']} - F1: {modelo['F1_Score']:.4f}, ROC: {modelo['ROC_AUC']:.4f}")

# Para prototipado rápido
rapidos = df_completo[
    (df_completo['Tiempo_Entrenamiento'] < 1.0) &
    (df_completo['F1_Score'] > df_completo['F1_Score'].quantile(0.7))
].sort_values('F1_Score', ascending=False)

print("\n- Para PROTOTIPADO RÁPIDO (< 1s, buen rendimiento):")
for _, modelo in rapidos.head(3).iterrows():
    print(f"   • {modelo['Nombre']} - F1: {modelo['F1_Score']:.4f}, Tiempo: {modelo['Tiempo_Entrenamiento']:.3f}s")

# Insights por categoría
print("\n- INSIGHTS POR CATEGORÍA:")
for categoria in df_completo['Categoria'].unique():
    cat_data = df_completo[df_completo['Categoria'] == categoria]
    mejor_cat = cat_data.sort_values('F1_Score', ascending=False).iloc[0]
    
    print(f"\n   - {categoria}:")
    print(f"      Mejor modelo: {mejor_cat['Nombre']} (F1: {mejor_cat['F1_Score']:.4f})")
    print(f"      Promedio categoría: F1 {cat_data['F1_Score'].mean():.4f}, Tiempo {cat_data['Tiempo_Entrenamiento'].mean():.3f}s")
    print(f"      Recomendación: {'Excelente para este dataset' if cat_data['F1_Score'].mean() > df_completo['F1_Score'].quantile(0.7) else 'Considerar otros algoritmos'}")

# Crear resumen final
resumen_final = {
    'total_modelos_evaluados': len(df_completo),
    'mejor_modelo_global': {
        'nombre': top_5_global.iloc[0]['Nombre'],
        'f1_score': top_5_global.iloc[0]['F1_Score'],
        'categoria': top_5_global.iloc[0]['Categoria']
    },
    'modelo_mas_eficiente': {
        'nombre': top_5_eficiencia.iloc[0]['Nombre'],
        'ratio_eficiencia': top_5_eficiencia.iloc[0]['Ratio_Eficiencia'],
        'categoria': top_5_eficiencia.iloc[0]['Categoria']
    },
    'categoria_mas_exitosa': distribucion_mejores.index[0],
    'tiempo_promedio_mejores': mejores_modelos['Tiempo_Entrenamiento'].mean(),
    'f1_score_range': [df_completo['F1_Score'].min(), df_completo['F1_Score'].max()]
}

# Guardar resumen
with open('../models/resumen_comparacion_completa_winsorized.json', 'w') as f:
    json.dump(resumen_final, f, indent=2)

# Guardar DataFrame completo
df_completo.to_csv('../models/comparacion_completa_todos_modelos_winsorized.csv', index=False)

print(f"\n/  ANÁLISIS COMPLETADO")
print(f"   - {len(df_completo)} modelos analizados")
print(f"   - Mejor F1-Score: {df_completo['F1_Score'].max():.4f}")
print(f"   - Mejor ratio eficiencia: {df_completo['Ratio_Eficiencia'].max():.1f}")
print(f"   - Resultados guardados en ../models/")

print("\n" + "="*100)
print("/ PRÓXIMOS PASOS RECOMENDADOS:")
print("   1. Implementar los top 3 modelos en producción")
print("   2. Hacer análisis con datos winsorizing para comparar")
print("   3. Optimizar hiperparámetros de los mejores modelos")
print("   4. Evaluar ensemble de los top 5 modelos")
print("="*100)


RECOMENDACIONES FINALES BASADAS EN ANÁLISIS COMPLETO

- TOP 5 MODELOS RECOMENDADOS (rendimiento absoluto):
   1. NN High Regularization (100,50) - F1: 0.7021 | ROC: 0.7590 | Tiempo: 33.037s
   2. NN Two Symmetric (80,80) - F1: 0.6969 | ROC: 0.7547 | Tiempo: 21.394s
   3. NN Deep Uniform (100,100,100) - F1: 0.6916 | ROC: 0.7537 | Tiempo: 47.334s
   4. NN Deep Decreasing (150,100,50) - F1: 0.6909 | ROC: 0.7570 | Tiempo: 65.588s
   5. NN Small (100) - F1: 0.6887 | ROC: 0.7468 | Tiempo: 12.185s

- TOP 5 MODELOS EFICIENTES (mejor ratio rendimiento/tiempo):
   1. KNN k=5 Uniform - Ratio: 28.4 | F1: 0.6551 | Tiempo: 0.022s
   2. Naive Bayes Complement - Ratio: 27.7 | F1: 0.6147 | Tiempo: 0.021s
   3. Naive Bayes Multinomial - Ratio: 27.6 | F1: 0.6407 | Tiempo: 0.022s
   4. KNN k=10 Uniform - Ratio: 18.5 | F1: 0.6357 | Tiempo: 0.033s
   5. KNN k=15 Distance - Ratio: 17.4 | F1: 0.6655 | Tiempo: 0.037s

- RECOMENDACIONES POR CASO DE USO:

- Para PRODUCCIÓN (buen rendimiento + rápido):

- Para M